In [1]:
import bz2

In [2]:
file = bz2.BZ2File('train.ft.txt.bz2')

In [3]:
file_lines = file.readlines()

In [4]:
file_lines = [x.decode('utf-8') for x in file_lines]

In [5]:
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in file_lines]

In [6]:
sentences = [x.split(' ', 1)[1][:-1].lower() for x in file_lines]

In [7]:
import re

In [8]:
for i in range(len(sentences)):
    sentences[i] = re.sub('\d','0',sentences[i])

In [9]:
for i in range(len(sentences)):
    if 'www.' in sentences[i] or 'http:' in sentences[i] or 'https:' in sentences[i] or '.com' in sentences[i]:
        sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", sentences[i])

In [10]:
print(f'{sentences[255]}:{labels[255]}')

more than thomas, not geared toward children as much: i bought this thinking it would be packed with thomas stories. turns out there are only ten, which is a typical thomas dvd. however, between each story, it also has interviews with parents and their kids about what they like about thomas. as i parent, i'm not interested in this, and i know that my 0 year old would rather watch thomas than a bunch of strangers any day. plus, the sing-along section has one song. one. we have other thomas videos that has at least 0, and they aren't special dvd's, just normal thomas collections. i wouldn't recommend this to anyone, if you are looking for solely thomas stories and can find other collections.:0


In [11]:
X = sentences
y = labels

In [12]:
import pandas as pd

In [13]:
reviews_df = pd.DataFrame.from_dict({'X':X,'y':y})

In [14]:
reviews_df.isna().sum()

X    0
y    0
dtype: int64

In [15]:
blanks = []
for idx,rev,lbl in reviews_df.itertuples():
    if type(rev) == str:
        if rev.isspace():
            blanks.append(idx)

In [16]:
blanks

[]

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.33,random_state=42)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
tfidf = TfidfVectorizer(max_df=0.95,min_df=0.12,stop_words='english')

In [21]:
X_train = tfidf.fit_transform(X_train)

In [22]:
X_train

<2412000x10 sparse matrix of type '<class 'numpy.float64'>'
	with 4452866 stored elements in Compressed Sparse Row format>

In [23]:
X_test = tfidf.fit_transform(X_test)

In [24]:
X_test

<1188000x10 sparse matrix of type '<class 'numpy.float64'>'
	with 2190615 stored elements in Compressed Sparse Row format>

In [25]:
reviews_df['y'].value_counts()

1    1800000
0    1800000
Name: y, dtype: int64

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
lr = LogisticRegression()

In [28]:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
predicted = lr.predict(X_test)

In [30]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [31]:
accuracy_score(y_true=y_test,y_pred=predicted)

0.629979797979798

In [32]:
print(classification_report(y_true=y_test,y_pred=predicted))

              precision    recall  f1-score   support

           0       0.60      0.76      0.67    594131
           1       0.67      0.50      0.58    593869

    accuracy                           0.63   1188000
   macro avg       0.64      0.63      0.62   1188000
weighted avg       0.64      0.63      0.62   1188000



In [33]:
print(confusion_matrix(y_true=y_test,y_pred=predicted))

[[449924 144207]
 [295377 298492]]


In [34]:
confusion_df = pd.DataFrame.from_dict({'0':[confusion_matrix(y_true=y_test,y_pred=predicted)[0][0],confusion_matrix(y_true=y_test,y_pred=predicted)[1][0]],'1':[confusion_matrix(y_true=y_test,y_pred=predicted)[0][1],confusion_matrix(y_true=y_test,y_pred=predicted)[1][1]]})

In [35]:
confusion_df.set_index(['0','1'],inplace=True)

In [36]:
confusion_df

,
0,1
449924,144207
295377,298492


In [37]:
import numpy as np

In [38]:
from sklearn.svm import LinearSVC

In [39]:
svm = LinearSVC()

In [40]:
svm.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [41]:
predicted = svm.predict(X_test)

In [42]:
accuracy_score(y_test,predicted)

0.6295934343434343

In [43]:
from sklearn.neighbors import KNeighborsClassifier

In [44]:
knn = KNeighborsClassifier()

In [45]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [47]:
predicted = knn.predict(X_test[:10])

In [48]:
accuracy_score(y_test[:10],predicted)

0.6